# Obiettivo del progetto
Dopo aver analizzato e compreso i dati, si vuole studiare meglio la correlazione riscontrata tra distanza e prezzo e si è individuato l'obiettivo del progetto. L'obiettivo è quello di verificare se c’è una stagionalità, nella quale i prezzi per alcuni mesi sono molto più elevati rispetto ad altri o se ci sono grandi variazioni di prezzo tra i diversi mesi rispetto alle diverse distanze.

## Descrizione del job proposto
Avendo a disposizione un solo file *.csv* si è pensato si usare un pattern di tipo *self-join*:

-	**Prima aggregazione**: aggregare per ogni combinazione di aeroporto di partenza e destinazione (*startingAeroport* e *destinationAeroport*) per ottenere la distanza media di viaggio (*totalTravelDistance*). A partire dalla distanza media generare una nuova colonna che indichi la fascia di distanza del volo (breve distanza, media distanza, lunga distanza);

-	**Join**: unire il dataset originale con il risultato ottenuto;

-	**Seconda aggregazione**: aggregare per fascia di distanza e mese (*flightDate*, da cui si ricava il mese) per ottenere per ciascuna combinazione il prezzo medio.

### Caricamento libreria Spark

Per prima cosa, si deve importare la libreria spark per avviare una `spark-shell`; in seguito verrà mostrato il link tramite il quale è possibile accedere all'interfaccia utente di Spark.

In [ ]:
import org.apache.spark

In [ ]:
// DO NOT EXECUTE - this is needed just to avoid showing errors in the following cells
val sc = spark.SparkContext.getOrCreate()

### Parser del file .csv

Nella cella sottostante è implementata una `case class Flight` con come parametri tutte le colonne(*) presenti nel file .csv descritto nel notebook [data-exploration.ipynb](./data-exploration.ipynb) e un `FlightParser` che consentente l'estrazione delle informazioni necessarie per popolare l'oggetto RDD di Spark.

(*) per risolvere il `job` proposto verranno utilizzate solo alcune delle colonne.

In [2]:
import java.text.SimpleDateFormat
import java.util.Calendar

/**
 * Flight case class.
 */
case class Flight(
    legId: String,
    searchMonth: Int,
    flightMonth: Int,
    startingAirport: String,
    destinationAirport: String,
    fareBasisCode: String,
    travelDuration: String,
    elapsedDays: Int,
    isBasicEconomy: Boolean,
    isRefundable: Boolean,
    isNonStop: Boolean,
    baseFare: Double,
    totalFare: Double,
    seatsRemaining: Int,
    totalTravelDistance: Double,
    segmentsDepartureTimeEpochSeconds: String,
    segmentsDepartureTimeRaw: String,
    segmentsArrivalTimeEpochSeconds: String,
    segmentsArrivalTimeRaw: String,
    segmentsArrivalAirportCode: String,
    segmentsDepartureAirportCode: String,
    segmentsAirlineName: String,
    segmentsAirlineCode: String,
    segmentsEquipmentDescription: String,
    segmentsDurationInSeconds: String,
    segmentsDistance: String,
    segmentsCabinCode: String
) extends Serializable

/**
 * Flight parser.
 */
object FlightParser extends Serializable {

  val comma = ","

  /**
   * Convert from date (String) to month (Int).
   * @param dateString the date
   * @return the month
   */
  def monthFromDate(dateString: String): Int = {
    val sdf = new SimpleDateFormat("yyyy-MM-dd")
    val date = sdf.parse(dateString.trim)
    val cal = Calendar.getInstance()
    cal.setTime(date)
    cal.get(Calendar.MONTH) + 1
  }

  /**
   * Function to parse flights records.
   * @param line that has to be parsed
   * @return Flight object, None in case of input errors
   */
  def parseFlightLine(line: String): Option[Flight] = {
    try {
      val columns = line.split(comma)
      Some(
        Flight(
          legId = columns(0).trim,
          searchMonth = monthFromDate(columns(1)),
          flightMonth = monthFromDate(columns(2)),
          startingAirport = columns(3).trim,
          destinationAirport = columns(4).trim,
          fareBasisCode = columns(5).trim,
          travelDuration = columns(6).trim,
          elapsedDays = columns(7).trim.toInt,
          isBasicEconomy = columns(8).trim.toBoolean,
          isRefundable = columns(9).trim.toBoolean,
          isNonStop = columns(10).trim.toBoolean,
          baseFare = columns(11).trim.toDouble,
          totalFare = columns(12).trim.toDouble,
          seatsRemaining = columns(13).trim.toInt,
          totalTravelDistance = columns(14).trim.toDouble,
          segmentsDepartureTimeEpochSeconds = columns(15).trim,
          segmentsDepartureTimeRaw = columns(16).trim,
          segmentsArrivalTimeEpochSeconds = columns(17).trim,
          segmentsArrivalTimeRaw = columns(18).trim,
          segmentsArrivalAirportCode = columns(19).trim,
          segmentsDepartureAirportCode = columns(20).trim,
          segmentsAirlineName = columns(21).trim,
          segmentsAirlineCode = columns(22).trim,
          segmentsEquipmentDescription = columns(23).trim,
          segmentsDurationInSeconds = columns(24).trim,
          segmentsDistance = columns(25).trim,
          segmentsCabinCode = columns(26).trim
        )
      )
    } catch {
      case e: Exception =>
        // println(s"Errore durante il parsing della riga '$line': ${e.getMessage}")
        None
    }
  }
}

import java.text.SimpleDateFormat
import java.util.Calendar
defined class Flight
defined object FlightParser


### Caricamento dei dati

Con la seguente cella si effettua il caricamento del file *itineraries-sample\<N\>.csv*, dove con N si intende la percentuale di dati campionati dal file originale di 31,09 GB.

I file disponibili sono scaricabili dalla cartella su [OneDrive](https://liveunibo-my.sharepoint.com/:f:/g/personal/giulia_nardicchia_studio_unibo_it/Ei2686kRO3JFrY-4LnImGpwBtge9FRErDnIgvT2h2QB-Pg?e=VrufWl) e hanno percentuale: `02`, `16` e `33`.

In [31]:
val datasetsPath = "../../../datasets/big/"
val fileName = "itineraries-sample16.csv"

val rawData = sc.textFile(datasetsPath + fileName)

datasetsPath: String = ../../../datasets/big/
fileName: String = itineraries-sample16.csv
rawData: org.apache.spark.rdd.RDD[String] = ../../../datasets/big/itineraries-sample16.csv MapPartitionsRDD[79] at textFile at <console>:43


Trasformazione di un RDD composto da dati grezzi (*rawData*) in un RDD di oggetti `Flight`. La funzione `FlightParser.parseFlightLine` analizza ogni riga. `flatMap` appiattisce i risultati, scartando automaticamente le righe non valide.

In [4]:
val rddFlights = rawData.flatMap(FlightParser.parseFlightLine)

rddFlights: org.apache.spark.rdd.RDD[Flight] = MapPartitionsRDD[2] at flatMap at <console>:28


Per verificare che non ci siano stati problemi di *parsing*, con la cella seguente si vuole eseguire un'azione. La funzione `count()` calcola il numero di righe valide.

In [5]:
rddFlights.count()

res0: Long = 1520662


### Prima aggregazione

Innanzitutto si utilizza `map` per ridurre i dati da manipolare, quindi vengono scartate tutte le colonne che non servono a svolgere il job proposto e per trasformare i dati di tipo (chiave, valore). Si vuole aggregare per ogni combinazione di aeroporto di partenza e destinazione (*startingAeroport* e *destinationAeroport*) per ottenere la distanza media di viaggio (*totalTravelDistance*). Per fare questo si utilizza la funzione `aggregateByKey()` passando come argomenti: (0.0, 0), i due valori iniziali, una funzione *combining* (*map*) per indicare come i singoli valori sono combinati con l'accumulatore e una funzione di *merging* (*reduce*) per indicare come gli accumulatori per le differenti partizioni devono essere uniti. Come funzioni di *combining* sono state passate due funzioni **associative** e **commutative** quali la somma e il conteggio, una volta ottenuti questi risultati si effettua il rapporto per calcolare la media delle distanze.

In [6]:
val avgDistances = rddFlights
  .map(flight => ((flight.startingAirport, flight.destinationAirport), flight.totalTravelDistance))
  .aggregateByKey((0.0, 0))(
    (acc, travelDistance) => (acc._1 + travelDistance, acc._2 + 1),
    (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
  )
  .mapValues { case (sumDistance, count) => sumDistance / count }

avgDistances: org.apache.spark.rdd.RDD[((String, String), Double)] = MapPartitionsRDD[5] at mapValues at <console>:33


In [7]:
avgDistances.collect()

res1: Array[((String, String), Double)] = Array(((BOS,LGA),406.6977958842578), ((IAD,ORD),841.525204359673), ((EWR,PHL),1039.5994575045208), ((DTW,LGA),694.1526669795088), ((OAK,DFW),2123.889001864091), ((ATL,DEN),1513.575124745888), ((IAD,CLT),587.9657102869139), ((DEN,LGA),1804.57909562639), ((DTW,EWR),736.2682451253482), ((LGA,DFW),1455.155069582505), ((OAK,JFK),3126.575707702436), ((DEN,DTW),1578.6580700623254), ((JFK,IAD),703.4175354183374), ((ORD,MIA),1521.1334047682828), ((IAD,DFW),1363.3681891954557), ((DEN,PHL),1852.40172900494), ((OAK,DEN),1419.471807628524), ((BOS,JFK),261.94046744083494), ((SFO,JFK),2652.746982695943), ((DTW,MIA),1462.8436163714111), ((PHL,OAK),2949.3589503280223), ((CLT,LGA),665.3443708609271), ((DTW,JFK),842.9147381242387), ((ATL,IAD),647.0074156470153), (...


A partire dalla distanza media, si intende generare una nuova colonna che specifichi la fascia di distanza del volo, distinguendo tra breve distanza, media distanza e lunga distanza.

Poiché usare valori numerici *hard coded* rappresenta una *bad practice*, si è deciso di utilizzare il minimo, il massimo e il numero di classi per calcolare dinamicamente l'intervallo delle fasce di distanza. Tuttavia, si è consapevoli che, nel caso in cui il codice venisse riutilizzato su un diverso dataset, gli intervalli calcolati potrebbero perdere significato. Le distanze, infatti, potrebbero variare significativamente, facendo sì che uno stesso percorso, classificato come breve in un dataset, risulti invece di media o lunga distanza in un altro.

Per ottenere il minimo e il massimo su `avgDistances` appena calcolato, si utilizza la funzione `aggregate()` specificando come valore iniziale per il minimo `Double.MaxValue` mentre per il massimo `Double.MinValue`. Per confrontare, invece, i valori tra di loro si utilizza il package `scala.math` con la funzione `math.min()` e `math.max()`.

In [8]:
val (minDistance, maxDistance) = avgDistances
    .aggregate((Double.MaxValue, Double.MinValue))(
        (acc, value) => (math.min(acc._1, value._2), math.max(acc._2, value._2)),
        (acc1, acc2) => (math.min(acc1._1, acc2._1), math.max(acc1._2, acc2._2))
    )

minDistance: Double = 185.0
maxDistance: Double = 3366.947416137806


In [9]:
val numClasses = 3
val range = (maxDistance - minDistance) / numClasses

numClasses: Int = 3
range: Double = 1060.649138712602


Per calcolare l'intervallo in maniera equidistante sono stati adottati i seguenti limiti:
- **Breve**: se la distanza media è inferiore a *minimo + intervallo*;
- **Media**: se la distanza media è compresa tra *[minimo + intervallo; minimo + 2 * intervallo)*;
- **Lunga**: se la distanza media è superiore a *minimo + (numero classi - 1) * intervallo*.

In [10]:
val classifiedDistances = avgDistances.mapValues {
  case (avgDistance) => 
     val classification = if (avgDistance < minDistance + range) "short"
     else if (avgDistance <= minDistance + (numClasses - 1) * range ) "medium"
     else "long"
    classification
}

classifiedDistances: org.apache.spark.rdd.RDD[((String, String), String)] = MapPartitionsRDD[6] at mapValues at <console>:30


In [11]:
classifiedDistances.collect()

res2: Array[((String, String), String)] = Array(((BOS,LGA),short), ((IAD,ORD),short), ((EWR,PHL),short), ((DTW,LGA),short), ((OAK,DFW),medium), ((ATL,DEN),medium), ((IAD,CLT),short), ((DEN,LGA),medium), ((DTW,EWR),short), ((LGA,DFW),medium), ((OAK,JFK),long), ((DEN,DTW),medium), ((JFK,IAD),short), ((ORD,MIA),medium), ((IAD,DFW),medium), ((DEN,PHL),medium), ((OAK,DEN),medium), ((BOS,JFK),short), ((SFO,JFK),long), ((DTW,MIA),medium), ((PHL,OAK),long), ((CLT,LGA),short), ((DTW,JFK),short), ((ATL,IAD),short), ((ATL,MIA),short), ((DTW,IAD),short), ((OAK,LGA),long), ((SFO,EWR),long), ((IAD,SFO),long), ((CLT,SFO),long), ((BOS,ATL),short), ((LAX,DEN),short), ((DEN,JFK),medium), ((BOS,LAX),long), ((SFO,IAD),long), ((DTW,DEN),medium), ((ORD,LGA),short), ((ATL,OAK),long), ((MIA,CLT),short), ((EWR,...


### Join + Seconda aggregazione

Si unisce il dataset originale con il risultato ottenuto, aggregando poi per fascia di distanza e mese (*flightDate*, da cui si ricava il mese) si ottiene per ciascuna combinazione il prezzo medio.

In [12]:
val resultJob = rddFlights
  .map(flight => ((flight.startingAirport, flight.destinationAirport), (flight.flightMonth, flight.totalFare)))
  .join(classifiedDistances)
  .map {
    case (_, ((flightMonth, totalFare), classification)) => ((flightMonth, classification), (totalFare, 1))
  }
  .reduceByKey((acc, totalFare) => (acc._1 + totalFare._1, acc._2 + totalFare._2))
  .map {
    case ((flightMonth, classification), (sumTotalFare, count)) => (flightMonth, classification, sumTotalFare / count)
  }

resultJob: org.apache.spark.rdd.RDD[(Int, String, Double)] = MapPartitionsRDD[13] at map at <console>:35


In [13]:
resultJob.collect()

res3: Array[(Int, String, Double)] = Array((9,long,405.4846256093827), (11,short,222.00556413449337), (11,long,382.2554902106178), (5,long,533.1691284153799), (7,short,290.6924974546775), (9,short,256.1263844227748), (4,long,480.8690961538476), (9,medium,293.22341323056406), (11,medium,254.47777319902355), (8,medium,322.54506223357464), (7,long,554.1280932802313), (6,long,598.8304861754743), (8,short,265.2235958197947), (5,medium,354.6517011517386), (10,short,259.31215532219113), (7,medium,382.5014410856043), (10,long,411.65239396939035), (5,short,278.2944923340169), (8,long,462.04696232908327), (6,short,296.05174334065094), (4,short,306.4335681610265), (6,medium,395.04683677556426), (10,medium,294.67349021285935), (4,medium,331.0029844885145))


## Job Not Optimized
L'ordine con cui sono state eseguite alcune operazioni, è migliorabile e quindi a partire dal codice scritto nelle precedenti celle, si è proceduto a *"rifattorizzare"* l'implementazione. Di seguito il *job* proposto non ottimizzato, in cui il nome delle variabili usate è il medesimo con l'aggiunta del suffisso NO (*Not Optimized*).

Differenze:

   - Innanzitutto, è stato migliorato il *mapping* iniziale, così da evitare di eseguire un ulteriore *mapping* in seguito per effettuare il *join*. In pratica vengono estratte le colonne a cui si è interessati subito (*totalTravelDistance, flightMonth, totalFare*) e viene usato quello come dataset di partenza.

   - Il *join*, adesso viene eseguito sul risultato ottenuto con la classificazione, al contrario della modalità precedente.

In [14]:
val numClassesNO = 3

val rddFlightsNO = rawData.flatMap(FlightParser.parseFlightLine)
    // (k,v) => (startingAirport, destinationAirport), (totalTravelDistance, flightDate, totalFare))
    .map(flight => ((flight.startingAirport, flight.destinationAirport),
                    (flight.totalTravelDistance, flight.flightMonth, flight.totalFare)))

val avgDistancesNO = rddFlightsNO
    .aggregateByKey((0.0, 0))(
        (acc, travelDistance) => (acc._1 + travelDistance._1, acc._2 + 1),
        (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
    )
    // (k,v) => ((startingAirport, destinationAirport), avgDistance)
    .mapValues { case (sumDistance, count) => sumDistance / count }

val (minDistanceNO, maxDistanceNO) = avgDistancesNO
    .aggregate((Double.MaxValue, Double.MinValue))(
        (acc, avgDistance) => (math.min(acc._1, avgDistance._2), math.max(acc._2, avgDistance._2)),
        (acc1, acc2) => (math.min(acc1._1, acc2._1), math.max(acc1._2, acc2._2))
    )

val rangeNO = (maxDistanceNO - minDistanceNO) / numClassesNO

val resultJobNotOptimized = avgDistancesNO
    .mapValues {
      case d if d < minDistanceNO + rangeNO => "short"
      case d if d < minDistanceNO + (numClassesNO - 1) * rangeNO => "medium"
      case _ => "long"
    } // (k,v) => ((startingAirport, destinationAirport), classification)
    .join(rddFlightsNO)
    .map { case (_, (classification, (_, month, totalFare))) => ((month, classification), (totalFare, 1)) }
    .reduceByKey((acc, totalFare) => (acc._1 + totalFare._1, acc._2 + totalFare._2))
    .map { case ((month, classification), (sumTotalFare : Double, count: Int)) => (month, classification, sumTotalFare / count) }

numClassesNO: Int = 3
rddFlightsNO: org.apache.spark.rdd.RDD[((String, String), (Double, Int, Double))] = MapPartitionsRDD[15] at map at <console>:32
avgDistancesNO: org.apache.spark.rdd.RDD[((String, String), Double)] = MapPartitionsRDD[17] at mapValues at <console>:41
minDistanceNO: Double = 185.0
maxDistanceNO: Double = 3366.947416137806
rangeNO: Double = 1060.649138712602
resultJobNotOptimized: org.apache.spark.rdd.RDD[(Int, String, Double)] = MapPartitionsRDD[24] at map at <console>:60


In [15]:
resultJobNotOptimized.collect()

res4: Array[(Int, String, Double)] = Array((9,long,405.4846256093826), (11,short,222.00556413449337), (11,long,382.2554902106179), (5,long,533.1691284153799), (7,short,290.69249745467766), (9,short,256.1263844227749), (4,long,480.8690961538476), (9,medium,293.22341323056395), (11,medium,254.4777731990232), (8,medium,322.545062233574), (7,long,554.1280932802313), (6,long,598.8304861754746), (8,short,265.223595819795), (5,medium,354.65170115173936), (10,short,259.3121553221906), (7,medium,382.50144108560403), (10,long,411.65239396939035), (5,short,278.294492334017), (8,long,462.04696232908344), (6,short,296.05174334065117), (4,short,306.4335681610265), (6,medium,395.0468367755642), (10,medium,294.67349021286043), (4,medium,331.0029844885145))


- Cache/Persist
- Repartition/PartitionBy
- Broadcast variable

# Considerazioni sulle ottimizzazioni

In questa sezione si vogliono fare alcune considerazioni riguardo le ottimizzazioni.

## Partitioning

Nel job proposto, le partizioni applicate di *default* sono 19 da 32MB ciascuno.

In locale, essendoci 4 core a disposizione, vengono assegnate 5 partizioni per core (circa).

In remoto, essendoci 6*2 = 12 core a disposizione, vengono assegnate 2 partizioni per core (circa).

In [17]:
println(rddFlightsNO.partitioner)
println(rddFlightsNO.partitions.length)
rddFlightsNO.mapPartitionsWithIndex((index, iter) => Iterator((index, iter.size))).collect().foreach(println)

None
19
(0,84320)
(1,84270)
(2,82909)
(3,80423)
(4,83522)
(5,83474)
(6,83326)
(7,74715)
(8,83102)
(9,83171)
(10,82888)
(11,82202)
(12,81915)
(13,82019)
(14,81801)
(15,81669)
(16,81562)
(17,80749)
(18,42625)


## Job Optimized

Una volta capite quali tecniche di ottimizzazione sono da adottare per migliorare le performance, sia in termini di CPU sia in termini di memoria, è stato riscritto il *main job* e si è ottenuto il seguente codice.

In [35]:
import org.apache.spark.storage.StorageLevel._
import org.apache.spark.HashPartitioner

val numClassesO = 3
val numPartitions = sc.defaultParallelism
val p = new HashPartitioner(numPartitions)

val rddFlightsO = rawData.flatMap(FlightParser.parseFlightLine)
    // (k,v) => (startingAirport, destinationAirport), (totalTravelDistance, flightDate, totalFare))
    .map(flight => ((flight.startingAirport, flight.destinationAirport), 
                    (flight.totalTravelDistance, flight.flightMonth, flight.totalFare)))
    .partitionBy(p)
    .persist(MEMORY_AND_DISK_SER)
    //.cache()

val avgDistancesO = rddFlightsO
    .aggregateByKey((0.0, 0))(
    (acc, travelDistance) => (acc._1 + travelDistance._1, acc._2 + 1),
    (acc1, acc2) => (acc1._1 + acc2._1, acc1._2 + acc2._2)
    )
    // (k,v) => ((startingAirport, destinationAirport), avgDistance)
    .mapValues { case (sumDistance, count) => sumDistance / count }
    .persist(MEMORY_AND_DISK_SER)
    //.cache()

val (minDistanceO, maxDistanceO) = sc.broadcast(avgDistancesO
    .aggregate((Double.MaxValue, Double.MinValue))(
        (acc, avgDistance) => (Math.min(acc._1, avgDistance._2), Math.max(acc._2, avgDistance._2)),
        (acc1, acc2) => (Math.min(acc1._1, acc2._1), Math.max(acc1._2, acc2._2))
    )
).value

val rangeO = (maxDistanceO - minDistanceO) / numClassesO

val resultJobOptimized = avgDistancesO
    .mapValues {
      case d if d < minDistanceNO + rangeNO => "short"
      case d if d < minDistanceNO + (numClassesNO - 1) * rangeNO => "medium"
      case _ => "long"
    } // (k,v) => ((startingAirport, destinationAirport), classification)
    .join(rddFlightsO)
    .map { case (_, (classification, (_, month, totalFare))) => ((month, classification), (totalFare, 1)) }
    .reduceByKey((acc, totalFare) => (acc._1 + totalFare._1, acc._2 + totalFare._2))
    .map { case ((month, classification), (sumTotalFare, count)) => (month, classification, sumTotalFare / count) }

import org.apache.spark.storage.StorageLevel._
import org.apache.spark.HashPartitioner
numClassesO: Int = 3
numPartitions: Int = 12
p: org.apache.spark.HashPartitioner = org.apache.spark.HashPartitioner@c
rddFlightsO: org.apache.spark.rdd.RDD[((String, String), (Double, Int, Double))] = ShuffledRDD[95] at partitionBy at <console>:65
avgDistancesO: org.apache.spark.rdd.RDD[((String, String), Double)] = MapPartitionsRDD[97] at mapValues at <console>:75
minDistanceO: Double = 185.0
maxDistanceO: Double = 3365.842672333445
rangeO: Double = 1060.280890777815
resultJobOptimized: org.apache.spark.rdd.RDD[(Int, String, Double)] = MapPartitionsRDD[104] at map at <console>:97


In [36]:
resultJobOptimized.collect()

res18: Array[(Int, String, Double)] = Array((4,medium,330.325343533702), (10,short,259.3394174082933), (6,short,295.91989592541563), (8,long,461.90870554105663), (8,short,265.0115462401657), (10,long,412.0195542900455), (5,short,278.92276971784855), (11,medium,252.94729750177794), (6,medium,394.8199324698205), (4,long,478.75647959185164), (11,short,221.6570366598475), (9,short,256.2727750906376), (9,medium,293.2709198209645), (6,long,598.472643079619), (7,long,553.9726197049043), (5,long,534.0684905212262), (10,medium,295.50240627503814), (8,medium,323.21841789452395), (9,long,404.56801307297786), (7,short,290.5630628313932), (11,long,380.16898388672274), (7,medium,382.2243478418637), (5,medium,355.28277571564473), (4,short,304.4165681595479))


In [37]:
rddFlightsO.mapPartitionsWithIndex((index, iter) => Iterator((index, iter.size))).collect().foreach(println)

(0,910143)
(1,842931)
(2,792917)
(3,1286794)
(4,1097736)
(5,1118030)
(6,1017238)
(7,950797)
(8,1146810)
(9,801914)
(10,1044567)
(11,1157260)


In [21]:
resultJobNotOptimized.sortBy(_._3, ascending = true).collect()

res9: Array[(Int, String, Double)] = Array((11,short,222.00556413449337), (11,medium,254.4777731990232), (9,short,256.1263844227749), (10,short,259.3121553221906), (8,short,265.223595819795), (5,short,278.294492334017), (7,short,290.69249745467766), (9,medium,293.22341323056395), (10,medium,294.67349021286043), (6,short,296.05174334065117), (4,short,306.4335681610265), (8,medium,322.545062233574), (4,medium,331.0029844885145), (5,medium,354.65170115173936), (11,long,382.2554902106179), (7,medium,382.50144108560403), (6,medium,395.0468367755642), (9,long,405.4846256093826), (10,long,411.65239396939035), (8,long,462.04696232908344), (4,long,480.8690961538476), (5,long,533.1691284153799), (7,long,554.1280932802313), (6,long,598.8304861754746))


In [22]:
resultJobOptimized.sortBy(_._3, ascending = true).collect()

res10: Array[(Int, String, Double)] = Array((11,short,222.00556413448837), (11,medium,254.4777731990195), (9,short,256.1263844227687), (10,short,259.3121553222084), (8,short,265.2235958197813), (5,short,278.29449233403625), (7,short,290.6924974546788), (9,medium,293.2234132305729), (10,medium,294.67349021288015), (6,short,296.0517433406561), (4,short,306.43356816102573), (8,medium,322.54506223357726), (4,medium,331.0029844885147), (5,medium,354.6517011517579), (11,long,382.25549021061255), (7,medium,382.5014410856088), (6,medium,395.0468367755694), (9,long,405.48462560937975), (10,long,411.6523939694063), (8,long,462.04696232906446), (4,long,480.8690961538468), (5,long,533.1691284153992), (7,long,554.1280932802221), (6,long,598.8304861754707))


Questa cella, serve a liberare la memoria ed è stata eseguita solo quando necessario per motivi di *debugging*.

In [29]:
sc.getPersistentRDDs.foreach(_._2.unpersist())

## Salvataggio dei risultati su file

Innanzitutto si importa la modalità di salvataggio `org.apache.spark.sql.SaveMode`, poi si creano le cartelle dentro al quale mettere i risultati.

In [10]:
import org.apache.spark.sql.SaveMode

val jobNotOptimized = "../../../output/jobNotOptimized"
val jobOptimized = "../../../output/jobOptimized"

import org.apache.spark.sql.SaveMode
jobNotOptimized: String = ../../../../output/jobNotOptimized
jobOptimized: String = ../../../../output/jobOptimized


A partire dall'*RDD* si applica la funzione `coalesce(1)` per ridurre il partizionamento a una sola partizione in modo da ottenere un singolo file. Per salvare il file, bisogna usare delle API che appartengono all'oggetto *DataFrame*, ecco perché viene effettuato un *mapping* con la funzione `toDF()`. E' poi possibile richiamare l'interfaccia `write` per salvare il contenuto del *DataFrame* nel formato *.csv*, con il metodo di salvataggio `SaveMode.Overwrite` (se il file esiste già al momento del salvataggio, viene sovrascritto), all'interno della cartella precedentemente specificata.

In [ ]:
resultJobNotOptimized
  .coalesce(1)
  .toDF().write.format("csv").mode(SaveMode.Overwrite).save(jobNotOptimized)

In [ ]:
resultJobOptimized
  .coalesce(1)
  .toDF().write.format("csv").mode(SaveMode.Overwrite).save(jobOptimized)